# Covid-19 Exploratory Data Analysis

## Covid-19 Dataset Understanding

In [ ]:
#!pip install folium

In [ ]:
!pip install plotly

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import math
import random 
from datetime import timedelta

import warnings
warnings.filterwarnings('ignore')

#color pallette
cnf = "#393e46"
dth = "#ff2e63"
rec = "#21bf73"
act = "#fe9801"

# Data Preparation

In [ ]:
import plotly as py
py.offline.init_notebook_mode(connected=True)

In [ ]:
df = pd.read_csv('covid_19_data_cleaned.csv',parse_dates=['Date'])

country_daywise = pd.read_csv('country_daywise.csv',parse_dates=['Date'])
countrywise = pd.read_csv('countrywise.csv')
daywise = pd.read_csv('daywise.csv',parse_dates=['Date'])


In [ ]:
df['Province/State'] = df['Province/State'].fillna("")
df.head()

In [ ]:
countrywise

In [ ]:
confirmed = df.groupby('Date').sum()['Confirmed'].reset_index()

In [ ]:
confirmed

In [ ]:
recovered = df.groupby('Date').sum()['Recovered'].reset_index()
deaths = df.groupby('Date').sum()['Deaths'].reset_index()

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.query('Country== "US"')

In [ ]:
recovered.tail()

In [ ]:
deaths.tail()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=confirmed['Date'],y=confirmed['Confirmed'],
              mode = 'lines + markers',name = 'Confirmed',line = dict(color='Orange'
                                                                ,width=2)))

fig.add_trace(go.Scatter(x=recovered['Date'],y=recovered['Recovered'],
              mode = 'lines + markers',name = 'Recovered',line = dict(color='Green'
                                                                     ,width=2)))
fig.add_trace(go.Scatter(x=deaths['Date'],y=deaths['Deaths'],
              mode = 'lines + markers',name = 'Deaths',line = dict(color='Blue'
                                                                     ,width=2)))
fig.update_layout(title='Worldwide Covid-19 Cases')
fig.show()

# Cases Density Animation on World Map

In [ ]:
df.info()

In [ ]:
df['Date'] = df['Date'].astype(str)

In [ ]:
fig = px.density_mapbox(df,lat='Lat',lon='Long',hover_name='Country',
                       hover_data = ['Confirmed','Recovered','Deaths'],
                       animation_frame = 'Date',color_continuous_scale='Portland',
                       radius=7,zoom=0,height=500)
fig.update_layout(title='Worldwide Covid-19 Cases with Time Laps')
fig.update_layout(mapbox_style='open-street-map',mapbox_center_lon=0)
fig.show()

# Total Cases on Ships

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
ship_rows = df['Province/State'].str.contains('Grand Princess')
df[ship_rows]

# Cases Over the Time with Area Plot

In [ ]:
temp = df.groupby('Date')['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()
temp = temp[temp['Date']==max(temp['Date'])].reset_index(drop = True)

tm = temp.melt(id_vars = 'Date', value_vars = ['Active', 'Deaths', 'Recovered'])
fig = px.treemap(tm, path = ['variable'], values = 'value', height = 250, width = 800, color_discrete_sequence=[act, rec, dth])

fig.data[0].textinfo = 'label+text+value'
fig.show()

In [ ]:
temp = df.groupby('Date')['Recovered','Deaths','Active'].sum().reset_index()
temp = temp.melt(id_vars = 'Date',value_vars = ['Recovered','Deaths','Active'],
                var_name = 'Case',value_name = 'Count')
fig = px.area(temp,x='Date',y='Count',color='Case',height = 600,title = 'Cases over time',
             color_discrete_sequence = [rec,dth,act])
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

# Folium Maps

In [ ]:
temp = df[df['Date']==max(df['Date'])]

m = folium.Map(location=[0,0],tiles='cartodbpositron',min_zoom=1,max_zoom=4,
              zoom_start=1)
for i in range(0,len(temp)):
    folium.Circle(location=[temp.iloc[i]['Lat'],temp.iloc[i]['Long']],
                  color='crimson',fill='crimson',
                  tooltip = '<li><bold> Country: '+str(temp.iloc[i]['Country'])+
                 '<li><bold> Province: '+str(temp.iloc[i]['Province/State'])+
                 '<li><bold> Confirmed: '+str(temp.iloc[i]['Confirmed'])+
                 '<li><bold> Deaths: '+str(temp.iloc[i]['Deaths']),
                 radius = int(temp.iloc[i]['Confirmed'])*0.5).add_to(m)
m

# Confirmed Cases with Choropleth Map

In [ ]:
fig = px.choropleth(country_daywise,locations = 'Country',
                     locationmode='country names',
color=np.log(country_daywise['Confirmed']),hover_name='Country',
                     animation_frame=country_daywise['Date'].
                     dt.strftime('%Y-%m-%d'),title='Cases over time',
                    color_continuous_scale=px.colors.sequential.Inferno)
fig.update(layout_coloraxis_showscale=True)
fig.show()

# Deaths and Recoveries per 100 Cases

In [ ]:
daywise.head()

In [ ]:
fig_c = px.bar(daywise,x='Date',y='Confirmed',color_discrete_sequence=[act])
fig_d = px.bar(daywise,x='Date',y='Confirmed',color_discrete_sequence=[dth])

fig = make_subplots(rows=1,cols=2,shared_xaxes=False,horizontal_spacing=0.1,
                   subplot_titles=('Confirmed Cases','Deaths Cases'))
fig.add_trace(fig_c['data'][0],row=1,col=1)
fig.add_trace(fig_d['data'][0],row=1,col=2)

fig.update_layout(height=400)
fig.show()

# Confirmed and Death Cases with Static ColorMap

In [ ]:
fig_c = px.choropleth(countrywise,locations='Country',locationmode='country names',
                    color=np.log(countrywise['Confirmed']),hover_name='Country',
                    hover_data=['Confirmed'])
temp=countrywise[countrywise['Deaths']>0]
fig_d = px.choropleth(temp,locations='Country',locationmode='country names',
                    color=np.log(temp['Deaths']),hover_name='Country',
                    hover_data=['Deaths'])
fig = make_subplots(rows=1,cols=2,subplot_titles=['Confirmed','Deaths'],
                   specs=[[{'type':'choropleth'},{'type':'choropleth'}]])
fig.add_trace(fig_c['data'][0],row=1,col=1)
fig.add_trace(fig_d['data'][0],row=1,col=2)
fig.update(layout_coloraxis_showscale=False)

fig.show()

In [ ]:
fig1 = px.line(daywise,x='Date',y='Deaths / 100 Cases',color_discrete_sequence=[dth])
fig2 = px.line(daywise,x='Date',y='Recovered / 100 Cases',color_discrete_sequence=[dth])
fig3 = px.line(daywise,x='Date',y='Deaths / 100 Recovered',color_discrete_sequence=[dth])

fig = make_subplots(rows=1,cols=3,shared_xaxes=False,
                   subplot_titles=('Deaths / 100 Cases','Recovered / 100 Cases','Deaths / 100 Recovered'))

fig.add_trace(fig1['data'][0],row=1,col=1)
fig.add_trace(fig2['data'][0],row=1,col=2)
fig.add_trace(fig3['data'][0],row=1,col=3)

fig.update_layout(height=400)
fig.show()

# New Cases and No.of Countries

In [ ]:
fig_c = px.bar(daywise,x='Date',y='Confirmed',color_discrete_sequence=[act])
fig_d = px.bar(daywise,x='Date',y='No. of Countries',color_discrete_sequence=[dth])

fig = make_subplots(rows=1,cols=2,shared_xaxes=False, horizontal_spacing=0.1,
                   subplot_titles=('No. of New Cases per Day','No. of Countries'))
fig.add_trace(fig_c['data'][0],row=1,col=1)
fig.add_trace(fig_d['data'][0],row=1,col=2)
fig.show()


# Top 15 Countries Case Analysis

In [ ]:
top = 15
fig_c = px.bar(countrywise.sort_values('Confirmed').tail(top),x = 'Confirmed'
              ,y='Country',text='Confirmed',orientation='h',
               color_discrete_sequence=[cnf])
fig_d = px.bar(countrywise.sort_values('Confirmed').tail(top),x = 'Confirmed'
              ,y='Country',text='Deaths',orientation='h',
               color_discrete_sequence=[dth]) 

fig_a = px.bar(countrywise.sort_values('Active').tail(top),x = 'Active'
              ,y='Country',text='Active',orientation='h',
               color_discrete_sequence=[act])
fig_r = px.bar(countrywise.sort_values('Recovered').tail(top),x = 'Recovered'
              ,y='Country',text='Recovered',orientation='h',
               color_discrete_sequence=[rec]) 
fig_dc = px.bar(countrywise.sort_values('Deaths / 100 Cases').tail(top),x = 'Deaths / 100 Cases'
              ,y='Country',text='Deaths / 100 Cases',orientation='h',
               color_discrete_sequence=[rec]) 
fig_rc = px.bar(countrywise.sort_values('Recovered / 100 Cases').tail(top),x = 'Recovered / 100 Cases'
              ,y='Country',text='Recovered / 100 Cases',orientation='h',
               color_discrete_sequence=[rec]) 

fig = make_subplots(rows=5,cols=2,shared_xaxes=False,horizontal_spacing=0.25,
                   vertical_spacing=.1,
                   subplot_titles=('Confirmed Cases','Deaths Reported',
                                   'Recovered Cases' ,'Active Cases',
                                  'Deaths / 100 Cases','Recovered / 100 Cases'))

fig.add_trace(fig_c['data'][0],row=1,col=1)
fig.add_trace(fig_d['data'][0],row=1,col=2)
fig.add_trace(fig_r['data'][0],row=2,col=1)
fig.add_trace(fig_a['data'][0],row=2,col=2)
fig.add_trace(fig_dc['data'][0],row=3,col=1)
fig.add_trace(fig_rc['data'][0],row=3,col=2)
fig.update_layout(height=3000)
fig.show()



# Scatter Plot for Deaths vs Confirmed Cases

In [ ]:
top = 15
fig = px.scatter(countrywise.sort_values('Deaths',ascending=False).head(top),
                 x='Confirmed',y='Deaths',color='Country',size='Confirmed',
                height=600,text='Country',log_x=True,title='Deaths vs Confirmed Cases')
fig.update_traces(textposition='top center')
fig.update_layout(showlegend=False)
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

# Confirmed,Deaths,New Cases vs Country and Date

# Bar Plot

In [ ]:
#fig = px.bar(country_daywise,x='Date',y='Confirmed',color='Country',height=600,
            #title='Confirmed',color_discrete_sequence=px.colors.cyclical.mygbm)
#fig.show()

In [ ]:
gt_100 = country_daywise[country_daywise['Confirmed']>100]['Country'].unique()
temp = df[df['Country'].isin(gt_100)]

temp = temp.groupby(['Country','Date'])['Confirmed'].sum().reset_index()
temp = temp[temp['Confirmed']>100]

min_date = temp.groupby('Country')['Date'].min().reset_index()
min_date.columns = ['Country','Min Date']
min_date



# Tree Map Analysis

# Confirmed Cases

In [ ]:
full_latest = df[df['Date']== max(df['Date'])]

fig = px.treemap(full_latest.sort_values(by='Confirmed',ascending=False).reset_index
                (drop=True),path=['Country','Province/State'],values='Confirmed',
                height=700,title='Number of Confirmed Cases',
                color_discrete_sequence=px.colors.qualitative.Dark2)
fig.data[0].textinfo = 'label+text+value'
fig.show()

# Covid-19 vs Other Similar Epidemics

In [ ]:
epidemics = pd.DataFrame({
    'epidemic' : ['COVID-19', 'SARS', 'EBOLA', 'MERS', 'H1N1'],
    'start_year' : [2019, 2002, 2013, 2012, 2009],
    'end_year' : [2020, 2004, 2016, 2020, 2010],
    'confirmed' : [full_latest['Confirmed'].sum(), 8422, 28646, 2519, 6724149],
    'deaths' : [full_latest['Deaths'].sum(), 813, 11323, 866, 19654]
})

epidemics['mortality'] = round((epidemics['deaths']/epidemics['confirmed'])*100, 2)

epidemics.head()

In [ ]:
temp = epidemics.melt(id_vars='epidemic',value_vars=['confirmed','deaths','mortality'],
                     var_name='Case',value_name='Value')
fig = px.bar(temp,x='epidemic',y='Value',color='epidemic',text='Value',facet_col='Case',
            color_discrete_sequence=px.colors.qualitative.Bold)

fig.update_traces(textposition='outside')
fig.update_layout(uniformtext_minsize=8,uniformtext_mode='hide')
fig.update_yaxes(showticklabels=False)
fig.layout.yaxis2.update(matches=None)
fig.layout.yaxis3.update(matches=None)
fig.show()